In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Set up Chrome options
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run in headless mode for automation
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

# Initialize the Chrome driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Read the CSV file into a DataFrame
# csv_file_path = 'updated_csv_file1.csv'
csv_file_path = 'C:/Users/Hp/Downloads/field-nation-report-7-15-2024.csv'
df = pd.read_csv(csv_file_path)

# Add new columns to the DataFrame
df['Where Job is performed'] = ''
df['Address'] = ''

base_url = 'https://app.fieldnation.com/workorders/'

# Define the cookies
cookies = {
    '_gcl_au': '1.1.2061555482.1719596558',
    'hubspotutk': '4ace82fd91b1b36597e0dd3e2652ba44',
    'drift_aid': '1b152144-911b-4c6d-b0ec-2375680285b6',
    'driftt_aid': '1b152144-911b-4c6d-b0ec-2375680285b6',
    '_hjSessionUser_3860837': 'eyJpZCI6ImY0NjE5ZDJkLTJiOTQtNWNjNi1iYWJkLTUzNTRkOWU4YzcwNyIsImNyZWF0ZWQiOjE3MTk1OTY1NTg2MDYsImV4aXN0aW5nIjp0cnVlfQ==',
    'interval': '86400',
    'intercom-device-id-q77i1otv': '5cc23db0-1f1c-430a-b2f6-69ff8adfb9cb',
    'amp_c12f4f': 'e1cBwRXDITDPJkI6qYEcHn...1i1nr3q72.1i1nst392.5.0.5',
    'fnsid': 's%3Ai6KjROsELcnA6zXSFgiByvO4-iqXfTu1.me3gicfXPHsLCA9hnH0iczMuZEkmDHJQCC9Or%2BDqgig',
    'last_seen': '1719862844220',
    '_gid': 'GA1.2.935597310.1720521868',
    '_rdt_uuid': '1719596558268.1d0fc7ed-b20d-4bd3-9baf-0bb5534f22ad',
    '_clck': 'ihr69j%7C2%7Cfnb%7C0%7C1640',
    '__hstc': '263070099.4ace82fd91b1b36597e0dd3e2652ba44.1719596558236.1719955178686.1720521922992.4',
    '_ga': 'GA1.2.2037770958.1719596558',
    '_ga_XHDRZ3M33R': 'GS1.1.1720521920.6.0.1720521933.47.0.0',
    'FNSESS': 'MTo4Nzc1NDoxOjIwMTEyOjVjOGZiZmUyLWU5ZTktNDg1ZC04ODI0LTMwYTAyMmZiZDZjZjowOjU2ZGJjMjAzNjJhNjBmODkxODkzYzdiYTQ0NzZiNTdj',
    'ambassador_session.authentication-prod-oauth-filter.default': 'b77a5ff91140720aa1ff5320a13a2d48b66c2f7c239b3bbe913dbf628c54b93c',
    'ambassador_xsrf.authentication-prod-oauth-filter.default': 'edc7f84279309b3690288b1be705fbbfcf98ea2214ad8392066bcc949fc1ae93',
    '__cfruid': '75509c0171e4c126cb97acabf223acd829f62097-1721007839',
    'needs_to_accept_tos': 'false',
    'remindMeLater_87754': 'true',
    '_gat': '1',
    'intercom-session-q77i1otv': 'eWxpM2NJNlJiN0ZkS2xqUkZGOTJEcUx6ak5ZZEhrWGVTUjFHMFRhRTFXWHhiUzJTNHpaV3VHaFdkQjlRK0Jqai0tQjhDNkJNRzBjOFAxdVFhRFdJOUpwZz09--ad822e062367d18adfd4d8a87a9b4bdaee1d21d3',
}

# Navigate to the login page
login_url = 'https://id.fieldnation.com/login?client_id=5PUz4s3nEwH3&redirect_uri=https%3A%2F%2Fapp.fieldnation.com%2F.ambassador%2Foauth2%2Fredirection-endpoint&response_type=code&scope=openid&state=15b203a4d6c72cae04bbfad0e269a2e98ad2476fb5d28d05d8d6463942764c1d%3Ahttps%3A%2F%2Fapp.fieldnation.com%2Fworkorders%2F'
driver.get(login_url)


# Add cookies to the browser
for name, value in cookies.items():
    driver.add_cookie({'name': name, 'value': value, 'domain': '.fieldnation.com'})
driver.refresh()

for index, row in df.iterrows():
    print("index", index)
    work_order_id = row['Id']
    url = f"{base_url}{work_order_id}"
#     # Add cookies to the browser
#     for name, value in cookies.items():
#         driver.add_cookie({'name': name, 'value': value, 'domain': '.fieldnation.com'})
#     driver.refresh()
    driver.get(url)
   
    try:
        # Wait for the Location section to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'Tile__bd__3VIV6'))
        )

        # Get the page source and parse with BeautifulSoup
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        # Locate the Location section
        location_header = soup.find('h3', string="Location")
        if location_header:
            location_div = location_header.find_parent('div', class_='Tile__division__DZ80a')
            if location_div:
                location_details = location_div.find('div', {'data-nw-file': 'Location'})
                if location_details:
                    # Extract the location name
                    location_name = location_details.find('div', style='font-size: 1.6rem; line-height: 2.2rem;')
                    location_name = location_name.text.strip() if location_name else "N/A"

                    # Extract the address lines
                    address_line1 = location_details.find('div', {'data-nw-file': 'AddressLine1'})
                    address_line1 = address_line1.text.strip() if address_line1 else "N/A"

                    address_line2 = location_details.find('div', {'data-nw-file': 'AddressLine2'})
                    address_line2 = address_line2.text.strip() if address_line2 else "N/A"

                    address = f"{address_line1}\n{address_line2}"

                    # Update the DataFrame
                    df.at[index, 'Where Job is performed'] = location_name
                    df.at[index, 'Address'] = address

    except Exception as e:
        print(f"Error processing work order {work_order_id}: {e}")

# Save the updated DataFrame back to a CSV file
output_csv_file_path = 'updated_csv_file_with_addresses.csv'
df.to_csv(output_csv_file_path, index=False)

# Close the WebDriver
driver.quit()

print("CSV file has been updated and saved to", output_csv_file_path)


index 0
index 1
index 2
index 3
index 4
index 5
index 6
index 7
index 8
index 9
index 10
index 11
index 12
index 13
index 14
index 15
index 16
index 17
index 18
index 19
index 20
index 21
index 22
index 23
index 24
index 25
index 26
index 27
index 28
index 29
index 30
index 31
index 32
index 33
index 34
index 35
Error processing work order 13808954: Message: 
Stacktrace:
	GetHandleVerifier [0x00C4C1C3+27395]
	(No symbol) [0x00BE3DC4]
	(No symbol) [0x00AE1B7F]
	(No symbol) [0x00B22C65]
	(No symbol) [0x00B22D3B]
	(No symbol) [0x00B5EC82]
	(No symbol) [0x00B439E4]
	(No symbol) [0x00B5CB24]
	(No symbol) [0x00B43736]
	(No symbol) [0x00B17541]
	(No symbol) [0x00B180BD]
	GetHandleVerifier [0x00F03A93+2876371]
	GetHandleVerifier [0x00F57F5D+3221661]
	GetHandleVerifier [0x00CCD634+556916]
	GetHandleVerifier [0x00CD474C+585868]
	(No symbol) [0x00BECE04]
	(No symbol) [0x00BE9818]
	(No symbol) [0x00BE99B7]
	(No symbol) [0x00BDBF0E]
	BaseThreadInitThunk [0x76CEFCC9+25]
	RtlGetAppContainerNamedObjec

index 678
index 679
index 680
index 681
index 682
index 683
index 684
index 685
index 686
index 687
index 688
index 689
index 690
index 691
index 692
index 693
index 694
index 695
index 696
index 697
index 698
index 699
index 700
index 701
index 702
index 703
index 704
index 705
index 706
index 707
index 708
index 709
index 710
index 711
index 712
index 713
index 714
index 715
index 716
index 717
index 718
index 719
index 720
index 721
index 722
index 723
index 724
index 725
index 726
index 727
index 728
index 729
index 730
index 731
index 732
index 733
index 734
index 735
index 736
index 737
index 738
index 739
index 740
index 741
index 742
index 743
index 744
index 745
index 746
index 747
index 748
index 749
index 750
index 751
index 752
index 753
index 754
index 755
index 756
index 757
index 758
index 759
index 760
index 761
index 762
index 763
index 764
index 765
index 766
index 767
index 768
index 769
index 770
index 771
index 772
index 773
index 774
index 775
index 776
index 777


index 1453
index 1454
index 1455
index 1456
index 1457
index 1458
index 1459
index 1460
index 1461
index 1462
index 1463
index 1464
index 1465
index 1466
index 1467
index 1468
index 1469
index 1470
index 1471
index 1472
index 1473
index 1474
index 1475
index 1476
index 1477
index 1478
index 1479
index 1480
index 1481
index 1482
index 1483
index 1484
index 1485
index 1486
index 1487
index 1488
index 1489
index 1490
index 1491
index 1492
index 1493
index 1494
index 1495
index 1496
index 1497
index 1498
index 1499
index 1500
index 1501
index 1502
index 1503
index 1504
index 1505
index 1506
index 1507
index 1508
index 1509
index 1510
index 1511
index 1512
index 1513
index 1514
index 1515
index 1516
index 1517
index 1518
index 1519
index 1520
index 1521
index 1522
index 1523
Error processing work order 2270110: Message: 
Stacktrace:
	GetHandleVerifier [0x00C4C1C3+27395]
	(No symbol) [0x00BE3DC4]
	(No symbol) [0x00AE1B7F]
	(No symbol) [0x00B22C65]
	(No symbol) [0x00B22D3B]
	(No symbol) [0x00